In [2]:
import os
import gymnasium as gym
import matplotlib.pyplot as plt 
# %matplotlib inline
# os.environ["SDL_VIDEODRIVER"] = "dummy"
# from IPython.display import clear_output

In [3]:
import time 

In [4]:
env = gym.make(
    "LunarLander-v2",
    continuous  = False,
    gravity = -10.0,
    enable_wind = False,
    wind_power = 15.0,
    turbulence_power = 1.5,
    render_mode='rgb_array')


Print Agent Information

In [5]:
print("Observation Space: ", format(env.observation_space))
print("Action Space       ", format(env.action_space))

Observation Space:  Box([-90.        -90.         -5.         -5.         -3.1415927  -5.
  -0.         -0.       ], [90.        90.         5.         5.         3.1415927  5.
  1.         1.       ], (8,), float32)
Action Space        Discrete(4)


In [6]:
observation, info = env.reset(seed=42)
# Number of steps you run the agent for 
num_steps = 1500

In [7]:
for steps in range(num_steps):
    # env_screen=env.render()    
    # action = my_intelligent_agent_fn(obs)  # this is where you would insert your policy
    action = env.action_space.sample() 
    
    #apply the action
    observation, reward, terminated, truncated, info = env.step(action)
    
    # plt.show()
    if terminated or truncated:
        observation, info = env.reset()
env.close()

In [ ]:
import time 